In [26]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import random
from IPython.display import IFrame
import config
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

# Welcome! Let me guide you through the steps: 

### 1. Get the right k
### 2. Write the recommendor! 
##

## For the first part, we collected 4,300 songs from disco, jazz, punk, metal, classic and a lot of Madonna (she's quite diverse). 
##

In [8]:
X = pd.read_csv("audio_df.cvs")
X

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.415,0.7480,4,-5.008,1,0.0481,0.00305,0.000000,0.0903,0.154,120.056,audio_features,4ltqfN12ohaVZdM6C45gMg,spotify:track:4ltqfN12ohaVZdM6C45gMg,https://api.spotify.com/v1/tracks/4ltqfN12ohaV...,https://api.spotify.com/v1/audio-analysis/4ltq...,258419,4
1,0.779,0.3720,9,-6.252,1,0.0284,0.19500,0.000324,0.7030,0.327,129.968,audio_features,11uXhnPkoGQtk3STW7ItHr,spotify:track:11uXhnPkoGQtk3STW7ItHr,https://api.spotify.com/v1/tracks/11uXhnPkoGQt...,https://api.spotify.com/v1/audio-analysis/11uX...,184973,4
2,0.555,0.4740,6,-11.706,0,0.0668,0.22400,0.004260,0.1620,0.262,134.890,audio_features,7jhxagwOk9y1HC5ciRfmkQ,spotify:track:7jhxagwOk9y1HC5ciRfmkQ,https://api.spotify.com/v1/tracks/7jhxagwOk9y1...,https://api.spotify.com/v1/audio-analysis/7jhx...,216160,4
3,0.634,0.6630,11,-6.818,1,0.0331,0.43300,0.124000,0.1020,0.600,146.053,audio_features,41P6Tnd8KIHqON0QIydx6a,spotify:track:41P6Tnd8KIHqON0QIydx6a,https://api.spotify.com/v1/tracks/41P6Tnd8KIHq...,https://api.spotify.com/v1/audio-analysis/41P6...,177533,4
4,0.677,0.5510,7,-7.963,1,0.0293,0.02060,0.269000,0.0880,0.246,106.954,audio_features,3ceTUPvI4JAuSwFWfeB7Ym,spotify:track:3ceTUPvI4JAuSwFWfeB7Ym,https://api.spotify.com/v1/tracks/3ceTUPvI4JAu...,https://api.spotify.com/v1/audio-analysis/3ceT...,142667,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3406,0.308,0.0844,9,-23.293,0,0.0546,0.97200,0.919000,0.2490,0.128,76.537,audio_features,3ByR9ilB4WYabWS29OaAFI,spotify:track:3ByR9ilB4WYabWS29OaAFI,https://api.spotify.com/v1/tracks/3ByR9ilB4WYa...,https://api.spotify.com/v1/audio-analysis/3ByR...,341573,5
3407,0.723,0.4480,0,-7.508,1,0.0600,0.59700,0.000001,0.1310,0.376,62.491,audio_features,3LKGxFJU1456pg0cljV01N,spotify:track:3LKGxFJU1456pg0cljV01N,https://api.spotify.com/v1/tracks/3LKGxFJU1456...,https://api.spotify.com/v1/audio-analysis/3LKG...,214425,4
3408,0.798,0.9090,11,-4.258,1,0.0450,0.23100,0.000000,0.0502,0.866,117.947,audio_features,6maKDCbPjXwJGSdRUSOYNs,spotify:track:6maKDCbPjXwJGSdRUSOYNs,https://api.spotify.com/v1/tracks/6maKDCbPjXwJ...,https://api.spotify.com/v1/audio-analysis/6maK...,257467,4
3409,0.615,0.4190,11,-8.799,0,0.0396,0.01730,0.505000,0.1350,0.216,76.996,audio_features,0SAyhJfdRcU1LmwU1HKW18,spotify:track:0SAyhJfdRcU1LmwU1HKW18,https://api.spotify.com/v1/tracks/0SAyhJfdRcU1...,https://api.spotify.com/v1/audio-analysis/0SAy...,330773,4


In [11]:
#Only selecting the features to scale: 
X = X[["danceability","energy","loudness","speechiness","acousticness",
    "instrumentalness","liveness","valence","tempo","duration_ms"]]

In [12]:
#Now I scale:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns = X.columns)

In [19]:
#I choose 5 clusters:
kmeans = KMeans(n_clusters=5, random_state=1234)
kmeans.fit(X_scaled_df)
clusters = kmeans.predict(X_scaled_df)

/Users/valeludelgado/anaconda3/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [20]:
X["cluster"] = clusters
X

/var/folders/2k/4xrmmln9749_gpnkt8_q2md80000gn/T/ipykernel_99971/310910030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X["cluster"] = clusters


,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,cluster
0,0.415,0.7480,-5.008,0.0481,0.00305,0.000000,0.0903,0.154,120.056,258419,3
1,0.779,0.3720,-6.252,0.0284,0.19500,0.000324,0.7030,0.327,129.968,184973,3
2,0.555,0.4740,-11.706,0.0668,0.22400,0.004260,0.1620,0.262,134.890,216160,2
3,0.634,0.6630,-6.818,0.0331,0.43300,0.124000,0.1020,0.600,146.053,177533,1
4,0.677,0.5510,-7.963,0.0293,0.02060,0.269000,0.0880,0.246,106.954,142667,1
...,...,...,...,...,...,...,...,...,...,...,...
3406,0.308,0.0844,-23.293,0.0546,0.97200,0.919000,0.2490,0.128,76.537,341573,0
3407,0.723,0.4480,-7.508,0.0600,0.59700,0.000001,0.1310,0.376,62.491,214425,2
3408,0.798,0.9090,-4.258,0.0450,0.23100,0.000000,0.0502,0.866,117.947,257467,1
3409,0.615,0.4190,-8.799,0.0396,0.01730,0.505000,0.1350,0.216,76.996,330773,1


In [41]:
#Let's add id to the dataframe 
df = pd.read_csv("audio_df.cvs")
merged_df = pd.concat([X, df['id']], axis=1)


### 2. Second part, we write the recommendor! 

In [102]:

billboard = pd.read_csv('billboard.csv')
user_song = input("Hello! Please enter a song: ")
user_song = user_song.lower()
billboard['Song_lower'] = billboard['title'].str.lower()
filtered_df = billboard[billboard['Song_lower'] != user_song] 
if user_song in billboard['Song_lower'].values:
    random_title = random.choice(filtered_df['title'])
    random_title = sp.search(q=random_title,limit=1,market="GB")
    track_id = random_title['tracks']["items"][0]["id"]
    print(f"Looks like your song is in the top 100, we recommend another one from that list: ")
    display(IFrame(src="https://open.spotify.com/embed/track/"+track_id,width="320",height="80",
    frameborder="0",
    allowtransparency="true",
    allow="encrypted-media",
    ))
else:
    answer = input("Would you like a recommendation? Please answer either yes or no: ")
    if answer == 'yes':
        results = sp.search(q=user_song,limit=1,market="GB")
        track_id = results['tracks']["items"][0]["id"]
        audio_features = pd.DataFrame(sp.audio_features(track_id)[0], index=[0])
        audio_features = audio_features.drop(columns=["type","id","uri","track_href","analysis_url","time_signature", "key", "mode"],axis=1)
        audio_features_scaled = scaler.transform(audio_features)
        df1 = merged_df[merged_df["cluster"]==kmeans.predict(audio_features_scaled)[0]].reset_index()
        recommendation = df1["id"][random.randint(0,len(df1))]
        print("Based on your answer we recommend you this:")
        display(IFrame(src="https://open.spotify.com/embed/track/"+recommendation,width="320",height="80",
        frameborder="0",
        allowtransparency="true",
        allow="encrypted-media",
        ))
    else:
        print("Okay")

Hello! Please enter a song: flowers
Looks like your song is in the top 100, we recommend another one from that list: 


#the first trial 

billboard = pd.read_csv('billboard.csv')
user_song = input("Hello! Please enter a song: ")
user_song = user_song.lower()
billboard['Song_lower'] = billboard['title'].str.lower()
filtered_df = billboard[billboard['Song_lower'] != user_song] 
if user_song in billboard['Song_lower'].values:
    random_title = random.choice(filtered_df['title'])
    print(f"Looks like your song is in the top 100, we recommend another one from that list: {random_title}")

else:
    answer = input("Would you like recommendations based on your song? Please answer either yes or no:")
    if answer == 'yes':
        results = sp.search(q=user_song,limit=1,market="GB")
        track_id = results['tracks']["items"][0]["id"]
        audio_features = pd.DataFrame(sp.audio_features(track_id)[0], index=[0])
        audio_features = audio_features.drop(columns=["type","id","uri","track_href","analysis_url","time_signature", "key", "mode"],axis=1)
        audio_features_scaled = scaler.transform(audio_features)
        df1 = merged_df[merged_df["cluster"]==kmeans.predict(audio_features_scaled)[0]].reset_index()
        recommendation = df1["id"][random.randint(0,len(df1))]
        print("Based on your answer we recommend this:")
        display(IFrame(src="https://open.spotify.com/embed/track/"+recommendation,width="320",height="80",
        frameborder="0",
        allowtransparency="true",
        allow="encrypted-media",
        ))
    else:
        print("Okay"